In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [3]:
## Load the trained model, scaler and oneho
model= load_model('model.h5')
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)
with open('onehot_encoder_geo.pkl','rb') as f:
    onehot_encoder_geo = pickle.load(f)

In [5]:
# Sample input data (modify based on your model's feature requirements)
sample = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
}

# Convert to DataFrame
sample_df = pd.DataFrame([sample])

# Encode 'Gender'
sample_df['Gender'] = label_encoder_gender.transform(sample_df['Gender'])

# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform(sample_df[['Geography']]).toarray()
geo_feature_names = onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_df = pd.DataFrame(geo_encoded, columns=geo_feature_names)

# Drop original and add encoded geography
sample_df = sample_df.drop('Geography', axis=1)
sample_df = pd.concat([sample_df, geo_df], axis=1)

# Scale input
scaled_input = scaler.transform(sample_df)

# Make prediction
prediction = model.predict(scaled_input)
prediction = prediction[0][0]
print("Prediction (probability):", prediction)
print("Predicted class:", (prediction > 0.5).astype(int))
if prediction > 0.5:
    print("Customer is likely to churn.")
else:
    print("Customer is unlikely to churn.")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction (probability): 0.018868601
Predicted class: 0
Customer is unlikely to churn.
